# Retrieving subsets of our data from NCBI's FTP site

In this notebook we show how to retrieve subsets from our data in VCF format from NCBI's FTP site. Our VCF content is compressed and has a `tabix` index.

`pytabix` takes advantage of the partial downloads available via `tabix` indexing over FTP. You can also use the `tabix` program directly from the command-line. In this notebook we will use `pytabix` to perform queries because many people prefer to do their analysis in Python.

*Note*: You cannot run this notebook at [Binder](https://mybinder.org/) because they [do not allow](https://github.com/binder-examples/getting-data#large-public-files) FTP connections.

In [ ]:
%pip install --quiet "urllib3<2" pytabix pysam ratelimit requests

# Fetching a range of variants

Using the code below we first tell `tabix` where to find the compressed VCF file, and then use the method `query` to retrieve the single variant at `905373` on chromosome 1. This requires starting the query at `905372` and ending on `905373`. Larger regions are also possible.

This is really all that is needed to get the basic data. Everything else is for display and formatting.

In [ ]:
import tabix

ftp_server='ftp://ftp.ncbi.nlm.nih.gov'
ftp_path='/snp/population_frequency/latest_release/'
ftp_vcf_path=ftp_path + 'freq.vcf.gz'
ftp_idx_path=ftp_path + 'freq.vcf.gz.tbi'
tb = tabix.open(ftp_server + ftp_vcf_path, ftp_server + ftp_idx_path)

records = tb.query('NC_000001.11', 905372, 905373)

for row in records:
    print(str(row))

# Fetching the header and metadata

As mentioned earlier, `pytabix` does not have the ability to retrieve the VCF headers. We need to parse the headers separately. Given the size of our FTP files, it would be too time and space consuming to do that. Instead, we will show you how to download a portion of the file that will allow us to parse the headers, but that is small enough that it only takes a very short time to obtain it.

To do this we use a small function `fetch_bytes` that retrieves just the first `N` kilobytes of data (set in the `MAX_BYTES` variable) of the compressed file.

In [ ]:
from ftplib import FTP
from io import BytesIO

MAX_BYTES = 1024 * 8


def fetch_bytes(ftp: FTP, ftp_file: str, max_bytes: int = MAX_BYTES) -> BytesIO:
    """
    Fetches at most max_bytes from the FTP file passed as input

    Adapted from https://stackoverflow.com/a/53144697
    """
    ftp_data = BytesIO()

    def data_size(biost: BytesIO):
        return biost.getbuffer().nbytes

    size = ftp.size(ftp_file)

    ftp.voidcmd('TYPE I')
    conn = ftp.transfercmd(f"RETR {ftp_file}", 0)

    try:
        while data_size(ftp_data) < max_bytes:
            buf = conn.recv(
                min(size - data_size(ftp_data), max_bytes))
            if not buf:
                break
            ftp_data.write(buf)
    finally:
        conn.close()
    try:
        ftp.voidresp()
    except:
        pass

    ftp_data.seek(0)
    return ftp_data

We can call `fetch_bytes` to retrieve enough data from our compressed VCF file to make sure we have the VCF header locally, which is all we need. The function puts all those bytes into a `BytesIO` object. We can then parse the object using gzip. 

Finally, we print the data obtained with `tabix` just below the headers so that it is clear what data correspons to what fields.

In [ ]:
from ftplib import FTP

import gzip
from sys import stdout

ftp = FTP('ftp.ncbi.nlm.nih.gov')
ftp.login()

ftpfile = fetch_bytes(ftp, ftp_vcf_path)

f = gzip.open(ftpfile,'rb')

# Now print the full headers
for l in f:
    l = l.decode().rstrip()
    if '##' in l:
        print(l)
    elif '#CHROM' in l:
        samples = l.split()[9:]
        print(l)
    else:
        break
f.close()

# Now we print the rows that `tabix` found based on our query
records = tb.query('NC_000001.11', 905372, 905373)
for row in records:
    print('\t'.join(row))
    
ftp.close()

You can observe that the `FORMAT` used is `AN:AC`. The meaning of each of those two acronyms is shown in the header:

* `AN`: _Total allele count for the population, including `REF`_
* `AC`: _Allele count for each `ALT` allele for the population_

The accessions, or biosample ids, of those populations are shown starting on the tenth column of the fifth line. Their frequency data in `AN:AC` format is shown under those headers starting on the 6th line.

You can check the names of the populations in the link provided in the header:

https://www.ncbi.nlm.nih.gov/biosample/?term=popfreq

In the remainder of this tutorial we show how to programmatically print those frequencies with the population names instead of their accessions. To achieve this, we first show how to retrieve population metadata and create a dictionary from biosample ids to population names. Then we use that dictionary and the code in this section to print the frequencies in an easier to understand format.

## Retrieving and using population metadata

In other notebooks we presented code to retrieve study and population metadata from our services and to index the populations by their biosample ids. For convenience, we repeat some of that code here, and also add a convenience function to map biosample ids to population names.

In [ ]:
import json
from ratelimit import limits
from requests import get, codes as http_code
from typing import Any, Dict, List


VarFreqMetadata = List[Dict[str, Any]]
MetadataDict = Dict[str, Dict[str, Any]]


@limits(calls=1, period=1)  # Only one call per second
def get_metadata() -> VarFreqMetadata:
    """
    Retrieve information that describes all studies and populations
    used by the frequency endpoints
    """
    METADATA_URL = ("https://api.ncbi.nlm.nih.gov/variation/v0/"
                    "metadata/frequency")

    reply = get(METADATA_URL)
    if reply.status_code != http_code.ok:
        raise Exception("Request failed: {}\n{}".format(
            reply.status_code, METADATA_URL))

    content_type = reply.headers['content-type']
    if content_type != 'application/json':
        raise Exception("Unexpected content type: {}\n{}".format(
            content_type, METADATA_URL))

    return reply.json()

In [ ]:
def build_accession_to_name_dict(pop_list: List[MetadataDict]) -> MetadataDict:
    """
    Constructs a dictionary of biosample ids to population names for each
    population (biosample) in the list of populations received as input.
    """
    result: MetadataDict = dict()
    for population in pop_list:
        result.update({population["biosample_id"]: population["name"]})

        if 'subs' in population:
            subs_dict = build_accession_to_name_dict(
                    population['subs'])
            if subs_dict:
                result.update(subs_dict)
    return result

def build_population_dictionary(metadata_orig: VarFreqMetadata) -> MetadataDict:
    """
    Constructs a dictionary of biosample ids to population names for each
    study (bioproject) listed in the frequency metadata received as input.
    """
    result: MetadataDict = dict()

    for study in metadata_orig:
        result.update(
            build_accession_to_name_dict(study["populations"]))

    return result

Now we can use these functions to first retrieve the population metadata and then to create a map from biosample ids to population names. Finally, we print the name of the population with accession `SAMN10492705` as an example.

In [ ]:
metadata = get_metadata()
pop_dict = build_population_dictionary(metadata)

print(pop_dict["SAMN10492705"])

## Displaying frequency data in a more user-friendly manner

We are finally ready to print the data in the VCF file.

First we translate the biosamples in the VCF file to their names:

In [ ]:
# Print header with population names
for biosample_id in samples:
    print(biosample_id, ': ', pop_dict[biosample_id])

Then we print the subset of data in the VCF file that we queried before in a more user-friendly manner

In [ ]:
# same query as before
data = tb.query('NC_000001.11', 905372, 905373)

for row_num, row in enumerate(data):
    print('Row: ', row_num)
    for column, biosample_id in enumerate(samples):
        print('\t', pop_dict[biosample_id])
        counts = row[9:][column]
        an, ac = counts.split(':')
        print('\t\t', 'Total allele count (including REF):', an)
        print('\t\t', 'Allele count for each ALT allele for this population:', ac)